<a href="https://colab.research.google.com/github/LexanderThakur/Cats_vs_Dogs_U/blob/main/CVDU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Todo**


1. Setup device agnostic code
2. Plot some images via code for practice


3. Convert Data to Tensors






## 1. Get Cats vs Dog Data

In [1]:
from pathlib import Path
import zipfile
import requests

data_path = Path("data/")


image_path = data_path / "cvd"


if image_path.is_dir():
    print("Already exists")
else:
    print("Downloading...")

    image_path.mkdir(parents=True,
                     exist_ok=True)

    with open(data_path / "cvd.zip","wb") as f:
        request=requests.get("https://github.com/LexanderThakur/Cats_vs_Dogs_U/raw/main/cvd.zip")

        f.write(request.content)

        with zipfile.ZipFile(data_path / "cvd.zip","r") as z:
            z.extractall(data_path)
            print("Done lesssgoo baby")

Downloading...
Done lesssgoo baby
